# 🔧 LangGraph Graph API 완전 정복 가이드

## 📚 개요

**LangGraph Graph API**는 마치 **모듈형 컴포넌트를 조립하듯이** 복잡한 AI 시스템을 만들 수 있게 해주는 강력한 도구입니다! 🧩

### 🏗️ Graph API가 특별한 이유

일반적인 프로그래밍에서는 코드가 위에서 아래로 순서대로 실행됩니다. 하지만 Graph API는 다릅니다:

- **🔀 유연한 흐름**: 상황에 따라 다른 경로로 실행
- **🤖 AI 에이전트 협력**: 여러 AI가 함께 문제 해결
- **🔄 반복과 루프**: 만족할 때까지 재시도
- **🎯 조건부 실행**: 똑똑한 의사결정

### 🎯 이 튜토리얼에서 배울 것들

1. **🌐 그래프 기본 개념** - State, Nodes, Edges가 뭔지 쉽게 이해하기
2. **📊 상태 관리** - 데이터를 어떻게 저장하고 관리하는지
3. **🔨 노드 심화 학습** - 실제 작업을 수행하는 함수들
4. **🔀 엣지와 라우팅** - 다음에 어디로 갈지 결정하는 방법
5. **🚀 고급 기능** - Send, Command 등 전문가 기능들

### 🎮 실생활 비유로 이해하기

**LangGraph**를 **회사 업무 시스템**에 비유해보세요:

```
🏢 회사 (Graph)
├── 📋 업무 현황판 (State) - 모든 정보가 기록되는 곳
├── 👥 직원들 (Nodes) - 실제 일을 처리하는 사람들  
└── 📞 소통 규칙 (Edges) - 누구에게 언제 일을 넘길지 정하는 규칙
```

### 💡 핵심 철학

> **"각자의 역할이 명확하고, 소통이 원활한 팀이 최고의 결과를 만든다"**
> 
> _노드는 작업을 수행하고, 엣지는 다음 단계를 결정한다_ 🎯

### 🚀 준비되셨나요?

이제 실제 예제를 통해 LangGraph의 강력한 세계로 들어가봅시다! 


## 환경 설정 🛠️

LangGraph 기능을 체험하기 전에 필요한 도구들을 준비해봅시다!

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

---

# Part 1: 그래프 기본 개념 🌐

## 1.1 그래프가 뭐길래? 🤔

LangGraph에서 말하는 **"그래프"**는 수학 그래프가 아니라 **"업무 흐름도"**라고 생각하시면 됩니다! 

### 🏭 공장 생산라인으로 이해하기

공장에서 제품을 만드는 과정을 떠올려보세요:

```
📦 원재료 → 🔨 가공1 → ⚙️ 가공2 → 📋 검수 → ✅ 완제품
```

LangGraph도 이와 똑같습니다:

- **📦 원재료** = 입력 데이터 (사용자 질문, 텍스트 등)
- **🔨 가공 과정** = 노드들 (AI 모델, 함수들)
- **📋 중간 결과** = 상태 (State)
- **✅ 완제품** = 최종 결과

### 🎯 그래프의 3가지 핵심 구성요소

#### 1️⃣ **State (상태) - 📋 작업 현황판**
- **실생활 비유**: 공장의 **생산 현황판**
- **역할**: 현재 진행 상황과 데이터를 모두 기록
- **예시**: 현재 처리 중인 메시지, 단계별 결과들

#### 2️⃣ **Nodes (노드) - 👷 작업자들**  
- **실생활 비유**: 각 **작업 스테이션의 직원들**
- **역할**: 실제 작업을 수행하는 함수들
- **예시**: AI 모델 호출, 데이터 가공, 결과 검증

#### 3️⃣ **Edges (엣지) - 🚛 운송 경로**
- **실생활 비유**: **컨베이어 벨트와 운송 경로**
- **역할**: 다음에 어느 작업자에게 보낼지 결정
- **예시**: "A 작업 완료 → B 작업으로 이동"

### 🔄 LangGraph의 똑똑한 실행 방식

일반적인 프로그램과 달리 LangGraph는 **Google의 Pregel 시스템**에서 영감을 받았습니다:

1. **⚡ Super-step**: 한 번에 하나의 작업 단계 실행
2. **🔀 병렬 처리**: 같은 단계의 여러 작업을 동시에 실행
3. **📝 순차 기록**: 각 단계의 결과를 차근차근 기록
4. **🎯 스마트 종료**: 모든 작업이 끝나면 자동으로 멈춤

### 💡 왜 이런 방식이 좋을까?

✅ **유연성**: 상황에 따라 다른 경로로 처리  
✅ **확장성**: 새로운 작업자(노드) 추가 용이  
✅ **디버깅**: 각 단계별 상태 추적 가능  
✅ **재사용**: 만들어진 그래프를 다른 곳에서도 활용

이제 실제 코드로 간단한 그래프를 만들어봅시다! 🚀

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


# 간단한 그래프 예제
class SimpleState(TypedDict):
    """Simple state with a single field"""

    message: str


# StateGraph 생성
graph_builder = StateGraph(SimpleState)


# 노드 함수 정의
def processor(state: SimpleState):
    """Process the message in the state"""
    # 노드는 현재 상태를 입력으로 받고 업데이트를 반환
    return {"message": state["message"] + " - processed"}


# 노드 추가
graph_builder.add_node("processor", processor)

# 엣지 추가 (진입점과 종료점)
graph_builder.add_edge(START, "processor")
graph_builder.add_edge("processor", END)

# 그래프 컴파일 - 필수!
graph = graph_builder.compile()

In [ ]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(graph)

In [ ]:
# 그래프 실행
result = graph.invoke({"message": "Hello LangGraph"})
print(f"결과: {result}")

### 1.2 그래프 컴파일 - 🏭 공장 가동 준비!

그래프를 만든 후에는 **반드시 컴파일**해야 합니다! 이는 마치 **공장 가동 전 최종 점검**과 같습니다.

#### 🔍 컴파일 과정에서 일어나는 일들

1. **🛠️ 구조 검증**: "모든 작업자가 제자리에 있나?"
   - 연결되지 않은 노드(고아 노드) 확인
   - 실행 경로가 제대로 이어져 있는지 검증

2. **⚙️ 설정 적용**: "특별한 옵션이 필요한가?"
   - 체크포인터: 중간 결과 저장소 설정
   - 브레이크포인트: 디버깅을 위한 중단점

3. **🚀 실행 준비**: "이제 언제든 가동할 수 있어!"
   - 최적화된 실행 가능한 객체 생성

#### 💡 컴파일 옵션들

- **💾 checkpointer**: 그래프 실행 중간에 상태를 저장 (메모리 기능!)
- **⏸️ interrupt_before**: 특정 노드 실행 **전**에 일시정지
- **⏹️ interrupt_after**: 특정 노드 실행 **후**에 일시정지

이 옵션들은 복잡한 시스템을 디버깅하거나 사용자 확인이 필요할 때 매우 유용합니다!

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# 체크포인터와 함께 컴파일
memory = InMemorySaver()

graph_with_memory = graph_builder.compile(
    checkpointer=memory,  # 상태 저장을 위한 체크포인터
    # interrupt_before=["processor"],  # 특정 노드 전에 중단 (디버깅용)
    # interrupt_after=["processor"],   # 특정 노드 후에 중단
)

print("✅ 체크포인터가 있는 그래프 컴파일 완료!")

---

# Part 2: State (상태) 심화 학습 📊

## 2.1 State가 뭐고 왜 중요할까? 🤔

**State(상태)**는 그래프의 **기억 저장소**입니다! 마치 **데이터베이스의 백업 파일**처럼 현재 진행 상황을 모두 기록해두는 곳이죠.

### 🏨 호텔 프런트 데스크로 이해하기

호텔 프런트 데스크를 생각해보세요:

- **📋 투숙객 대장** = State 스키마 (어떤 정보를 기록할지 정의)
- **✍️ 기록 방식** = 리듀서 함수 (정보를 어떻게 업데이트할지)
- **📝 실제 기록** = 현재 상태 값들

### 🎯 State 스키마 정의 방법 3가지

LangGraph에서는 State를 정의하는 세 가지 방법이 있습니다:

#### 1️⃣ **TypedDict 방식** (⭐ 가장 추천!)
```python
class MyState(TypedDict):
    name: str
    count: int
```
- **장점**: 가벼움, 빠름, 간단함
- **단점**: 기본값 설정이 번거로움
- **언제 사용**: 대부분의 경우

#### 2️⃣ **Dataclass 방식**
```python
@dataclass
class MyState:
    name: str = "기본값"
    count: int = 0
```
- **장점**: 기본값 설정 쉬움
- **단점**: TypedDict보다 약간 무거움
- **언제 사용**: 기본값이 많이 필요할 때

#### 3️⃣ **Pydantic 방식**
```python
class MyState(BaseModel):
    name: str = "기본값"
    count: int = 0
```
- **장점**: 강력한 유효성 검증
- **단점**: 가장 무거움, 복잡함
- **언제 사용**: 엄격한 데이터 검증이 필요할 때

### 💡 어떤 방식을 선택해야 할까?

**입문자라면 TypedDict**부터 시작하세요! 간단하고 직관적이며 성능도 우수합니다. 🚀

In [ ]:
from typing import Annotated, List
from dataclasses import dataclass, field
from pydantic import BaseModel


# 방법 1: TypedDict (가장 일반적, 성능 우수)
class TypedDictState(TypedDict):
    """State using TypedDict"""

    count: int
    items: List[str]


# 방법 2: dataclass (기본값 지원)
@dataclass
class DataclassState:
    """State using dataclass with default values"""

    count: int = 0
    items: List[str] = field(default_factory=list)


# 방법 3: Pydantic (재귀적 검증, 성능은 떨어짐)
class PydanticState(BaseModel):
    """State using Pydantic for validation"""

    count: int = 0
    items: List[str] = []


print("✅ 세 가지 방식의 State 스키마 정의 완료!")

## 2.2 Reducers (리듀서) - 📝 업데이트 규칙 관리자

**리듀서**는 "새로운 정보가 들어왔을 때 기존 정보와 어떻게 합칠까?"를 정하는 **똑똑한 규칙**입니다!

### 🏪 상점 재고 관리로 이해하기

상점에서 재고를 관리한다고 생각해보세요:

#### 📦 **기본 리듀서 (덮어쓰기)**
```python
재고_수량: int  # 새 값으로 완전히 교체
```
**예시**: "사과 10개" → "사과 15개" (10개는 사라지고 15개가 됨)

#### 📋 **add 리듀서 (추가)**
```python
주문_목록: Annotated[List[str], add]  # 기존 목록에 추가
```
**예시**: ["사과", "바나나"] + ["오렌지"] = ["사과", "바나나", "오렌지"]

#### 🏆 **커스텀 리듀서 (최대값 유지)**
```python
최고_판매량: Annotated[int, lambda old, new: max(old, new)]
```
**예시**: 기존 50개, 새로 30개 → 50개 유지 (더 큰 값 선택)

### 🎯 리듀서의 핵심 원리

1. **🔄 자동 병합**: 노드에서 업데이트할 때 자동으로 적절히 합쳐짐
2. **🛡️ 데이터 보호**: 실수로 중요한 데이터를 잃어버리는 것 방지
3. **🎨 유연성**: 상황에 맞는 업데이트 전략 선택 가능

### 💡 언제 어떤 리듀서를 쓸까?

- **기본 리듀서**: 설정값, 현재 상태 등 (예: 현재_온도)
- **add 리듀서**: 메시지, 로그, 히스토리 등 (예: 대화_내역)  
- **커스텀 리듀서**: 특별한 로직이 필요할 때 (예: 최고_점수)

다음 예제에서 리듀서들이 실제로 어떻게 동작하는지 확인해봅시다! 🚀

In [ ]:
from operator import add


# 기본 리듀서 vs 커스텀 리듀서
class StateWithReducers(TypedDict):
    """State with custom reducers"""

    # 기본 리듀서: 값을 덮어씀
    current_value: int

    # add 리듀서: 리스트를 연결
    history: Annotated[List[str], add]

    # 커스텀 리듀서: 최대값 유지
    max_value: Annotated[int, lambda x, y: max(x, y)]


# 그래프 생성
builder = StateGraph(StateWithReducers)


def update_values(state: StateWithReducers):
    """Update various state values"""
    return {
        "current_value": 10,  # 덮어쓰기
        "history": ["새 항목"],  # 추가
        "max_value": 15,  # 최대값과 비교
    }


builder.add_node("updater", update_values)
builder.add_edge(START, "updater")
builder.add_edge("updater", END)

graph = builder.compile()

# 그래프 시각화
visualize_graph(graph)

In [ ]:
# 초기 상태로 실행
initial_state = {"current_value": 5, "history": ["초기 항목"], "max_value": 20}

result = graph.invoke(initial_state)
print("\n리듀서 적용 결과:")
print(f"  current_value: 5 → {result['current_value']} (덮어쓰기)")
print(f"  history: {result['history']} (추가)")
print(f"  max_value: {result['max_value']} (최대값 유지)")

## 2.3 메시지 처리 - 💬 대화의 기억을 남기기

**대화형 AI**에서는 이전 대화 내용을 기억하는 것이 정말 중요하죠! LangGraph는 이를 위한 **특별한 메시지 관리 시스템**을 제공합니다.

### 📱 카카오톡 대화방으로 이해하기

카카오톡 대화방을 떠올려보세요:

- **💬 대화 내역**: 지금까지 주고받은 모든 메시지들
- **➕ 새 메시지**: 기존 대화에 추가되는 새로운 메시지  
- **📝 메시지 타입**: 텍스트, 이미지, 파일 등 다양한 형태

### 🎯 LangGraph의 메시지 시스템

#### 방법 1: `add_messages` 리듀서 사용 (직접 정의)
```python
class ChatState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    metadata: dict  # 부가 정보
```

#### 방법 2: `MessagesState` 상속 (⭐ 더 간단!)
```python
class ExtendedChatState(MessagesState):
    # messages는 이미 정의되어 있음!
    metadata: dict  # 필요한 것만 추가
```

### 🔍 메시지 타입들

- **🙋 HumanMessage**: 사용자가 보낸 메시지
- **🤖 AIMessage**: AI가 답변한 메시지  
- **🛠️ SystemMessage**: 시스템 지시사항
- **⚙️ ToolMessage**: 도구 실행 결과

### 💡 왜 이렇게 관리할까?

✅ **대화 맥락 유지**: 이전 대화를 참고해서 더 나은 답변  
✅ **자동 추가**: 새 메시지가 알아서 대화 목록에 추가됨  
✅ **타입 안전성**: 메시지 종류별로 적절한 처리  
✅ **디버깅 용이**: 대화 과정을 단계별로 추적 가능

실제 예제를 통해 메시지가 어떻게 관리되는지 확인해봅시다! 💬

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import MessagesState


# 방법 1: add_messages 리듀서 사용
class ChatState(TypedDict):
    """State with message handling"""

    messages: Annotated[List[AnyMessage], add_messages]
    metadata: dict


# 방법 2: MessagesState 상속 (더 간단)
class ExtendedChatState(MessagesState):
    """Extended state inheriting from MessagesState"""

    # messages 키는 이미 정의되어 있음
    metadata: dict

In [ ]:
from langchain_core.runnables import RunnableConfig

# 메시지 처리 예제
builder = StateGraph(ChatState)


def process_chat(state: ChatState):
    """Process chat messages"""
    # 마지막 메시지 접근
    last_message = state["messages"][-1]

    # 응답 생성
    response = AIMessage(content=f"{last_message.content}")

    return {
        "messages": [response],  # add_messages가 자동으로 추가
        "metadata": {"processed": True},
    }


builder.add_node("chat", process_chat)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

memory = InMemorySaver()

chat_graph = builder.compile(checkpointer=memory)

In [ ]:
config = RunnableConfig(configurable={"thread_id": "1"})

# 실행
result = chat_graph.invoke(
    {
        "messages": [HumanMessage(content="안녕하세요")],
        "metadata": {},
        "user_info": {"name": "사용자"},
    },
    config=config,
)

print("\n메시지 처리 결과:")
for msg in result["messages"]:
    print(f"  [{msg.__class__.__name__}]: {msg.content}")

## 2.4 다중 스키마 - 🎭 역할에 따른 다른 모습

때로는 **같은 데이터라도 보는 관점에 따라 다르게 표현**해야 할 때가 있습니다. 이럴 때 다중 스키마가 빛을 발합니다!

### 🏪 온라인 쇼핑몰로 이해하기

온라인 쇼핑몰에서 하나의 주문을 생각해보세요:

#### 👤 **고객이 보는 화면** (InputState)
```
주문번호: ORD-2024-001
상품: 스마트폰 갤럭시 S24
수량: 1개
```

#### 📦 **창고에서 보는 화면** (InternalState)  
```
SKU: PHONE_SAMSUNG_S24_128GB_BLACK
재고위치: A-3-15
픽업담당자: 김창고
```

#### 📊 **관리자가 보는 화면** (OutputState)
```
처리상태: 배송준비완료
예상배송일: 2024-01-15
고객만족도: 92%
```

### 🎯 다중 스키마의 핵심 개념

#### **InputState** - 🚪 입구에서 받는 정보
- 그래프가 시작할 때 **사용자가 제공하는 데이터**의 형태
- **예시**: 사용자 질문, 업로드한 파일, 설정값 등

#### **InternalState** - 🏭 내부에서 사용하는 정보  
- 그래프 **내부 노드들이 실제로 사용하는** 데이터 구조
- **예시**: 중간 처리 결과, 임시 데이터, 내부 상태값 등

#### **OutputState** - 🎁 최종 결과물
- 그래프가 **완료되었을 때 사용자에게 반환하는** 데이터 형태
- **예시**: 최종 답변, 처리 결과, 요약 정보 등

### 💡 왜 이렇게 복잡하게 나눌까?

✅ **보안**: 내부 처리 로직을 사용자에게 노출하지 않음  
✅ **단순화**: 사용자는 필요한 정보만 간단하게 제공  
✅ **유연성**: 내부 구조 변경해도 외부 인터페이스는 그대로  
✅ **명확성**: 입력/출력/내부 처리가 명확히 구분됨

실제 예제에서 이 세 가지 스키마가 어떻게 협력하는지 살펴봅시다! 🚀

In [ ]:
# 그래프의 입력 스키마
class InputState(TypedDict):
    """Input schema - what users provide"""

    user_input: str


# 그래프의 출력 스키마
class OutputState(TypedDict):
    """Output schema - what users receive"""

    graph_output: str


# 내부에서 사용되는 State
class InternalState(TypedDict):
    """Internal schema - full state for processing"""

    user_input: str
    internal_state: str


# Private State
class PrivateState(TypedDict):
    """Private schema - for internal node communication"""

    private_state: str

In [ ]:
# 다중 스키마를 사용하는 그래프
builder = StateGraph(
    InternalState,
    input_schema=InputState,  # 입력 스키마 지정
    output_schema=OutputState,  # 출력 스키마 지정
)


def node_1(state: InputState) -> InternalState:
    """First node - reads from InputState, writes to InternalState"""
    # InputState를 받지만 InternalState의 키에 쓸 수 있음
    print(f"node_1: {state}")
    return {
        "user_input": state["user_input"],
        "internal_state": state["user_input"] + "(node_1 에서 처리됨)",
    }


def node_2(state: InternalState) -> PrivateState:
    """Second node - reads from InternalState, writes to PrivateState"""
    print(f"node_2: {state}")
    # 새로운 state 채널(temp_data) 추가 가능
    return {"private_state": f"PRIVATE: {state['internal_state']} (node_2 에서 처리됨)"}


def node_3(state: PrivateState) -> OutputState:
    """Final node - reads from PrivateState, writes to OutputState"""
    print(f"node_3: {state}")
    return {"graph_output": f"최종 결과: {state['private_state']} (node_3 에서 처리됨)"}


# 노드와 엣지 추가
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

multi_schema_graph = builder.compile()

visualize_graph(multi_schema_graph)

# 실행 - 입력은 InputState 형식
result = multi_schema_graph.invoke({"user_input": "사용자 입력"})

# 출력은 OutputState 형식
print(f"\n다중 스키마 결과: {result}")

---

# Part 3: Nodes (노드) 심화 학습 🔨

## 3.1 노드가 뭐고 어떻게 만들까? 🤔

**노드(Node)**는 실제 일을 하는 **일꾼**입니다! 마치 공장에서 각자 맡은 일을 처리하는 **작업자**와 같죠.

### 🏭 공장 작업자로 이해하기

공장에서 작업자가 일하는 모습을 떠올려보세요:

- **📋 작업 지시서** = state (현재 작업 상황)
- **⚙️ 도구와 장비** = config (작업 설정)  
- **👷 작업자** = 노드 함수
- **📦 완성품** = 업데이트된 state

### 🎯 노드 함수가 받을 수 있는 재료들

노드 함수는 최대 3가지 재료를 받을 수 있습니다:

#### 1️⃣ **state** (필수 재료 🌟)
```python
def my_node(state: MyState):
    # 현재 상태를 받아서 처리
```
- **역할**: 현재 그래프의 상태 정보
- **내용**: 지금까지 처리된 모든 데이터

#### 2️⃣ **config** (설정 재료 ⚙️)
```python  
def my_node(state: MyState, config: RunnableConfig):
    # 상태 + 설정 정보를 받아서 처리
```
- **역할**: 실행 시점의 설정 정보
- **내용**: thread_id, 사용자 정의 설정값 등

#### 3️⃣ **runtime** (런타임 재료 🏃)
```python
def my_node(state: MyState, config: RunnableConfig, *, store: BaseStore):
    # 모든 정보를 받아서 처리
```
- **역할**: 런타임 컨텍스트와 추가 리소스
- **내용**: 데이터베이스, 캐시, 외부 서비스 등

### 💡 언제 어떤 재료를 사용할까?

- **state만**: 간단한 데이터 처리 (90% 이상의 경우)
- **state + config**: 사용자별 설정이나 세션 관리가 필요할 때
- **모든 재료**: 복잡한 시스템 통합이나 고급 기능이 필요할 때

### 🚀 노드 함수 작성 팁

✅ **동기/비동기 모두 OK**: `async def`도 지원!  
✅ **명확한 타입**: 입력과 출력 타입을 명시하면 디버깅이 쉬워짐  
✅ **작은 단위**: 하나의 노드는 하나의 명확한 작업만  
✅ **에러 처리**: 예외 상황도 미리 고려

이제 실제로 다양한 노드들을 만들어봅시다! 🛠️

### 🎛️ Config 스키마 - 노드의 개별 설정표

**ConfigSchema**는 각 노드가 받을 수 있는 **특별한 설정값들의 목록**입니다. 마치 **음향 장비의 이퀄라이저**처럼 세부 조정이 가능해요!

In [ ]:
from typing_extensions import TypedDict


class ConfigSchema(TypedDict):
    my_runtime_value: str
    another_setting: bool


class MyState(MessagesState):
    pass

### 📡 Config로 노드에 정보 전달하기

**config 매개변수**는 노드가 실행될 때 **추가 정보를 받을 수 있는 통로**입니다!

#### 🎛️ 제어판으로 이해하기
- **🕹️ config**: 실행 중에 설정을 조정할 수 있는 제어판
- **⚙️ configurable**: 실제로 조정할 수 있는 설정값들
- **🎯 RunnableConfig**: 모든 설정 정보가 담긴 전체 패키지

config를 통해 **thread_id**(세션 구분), **사용자 정의 값** 등을 받을 수 있어요!

In [ ]:
from langchain_core.runnables import RunnableConfig


# state 와 config 를 받는 노드 함수 정의
def my_node(state: MyState, config: RunnableConfig):
    """
    Example node function that demonstrates how to access custom configuration values.

    Args:
        state (MyState): The current state of the graph.
        config (RunnableConfig): Configuration object containing runtime and user-defined settings.

    Returns:
        dict: Updated state dictionary.
    """
    # config["configurable"] 딕셔너리에서 사용자 정의 설정값을 가져옵니다.
    runtime_value = config["configurable"].get("runtime_value", "")
    setting_value = config["configurable"].get("setting_value", "")

    # runtime_value 값이 있으면 해당 값을 출력합니다.
    if runtime_value:
        print(f"runtime_value: {runtime_value}")

    # setting_value 값이 있으면 해당 값을 출력합니다.
    if setting_value:
        print(f"setting_value: {setting_value}")

    # 현재 state 값을 출력합니다.
    print(f"state: {state}")

    # 새로운 키와 값을 포함하는 상태를 반환합니다.
    return {"updated_key": "new_value"}

### 🔧 그래프에 Config 스키마 연결하기

**context_schema**를 지정하면 LangGraph가 **설정값을 자동으로 검증**해줍니다!

#### ✅ 이렇게 하면 좋은 점들:
- **오타 방지**: 잘못된 설정 키 이름을 미리 잡아냄
- **타입 안전**: 숫자가 들어가야 할 곳에 문자가 들어가면 경고
- **개발 편의**: IDE에서 자동완성 지원
- **문서화**: 어떤 설정을 받는지 명확히 표시

In [ ]:
from langgraph.graph import StateGraph

builder = StateGraph(MyState, context_schema=ConfigSchema)
builder.add_node("my_node", my_node)
builder.add_edge(START, "my_node")
builder.add_edge("my_node", END)

graph = builder.compile()

### 🚀 실제로 Config 값 전달하기

이제 그래프를 실행할 때 **원하는 설정값을 전달**해봅시다!

#### 📦 설정값 전달 방법
```python
graph.invoke(
    {"messages": [...]},  # 일반 입력 데이터
    config={
        "configurable": {    # 🎯 여기에 우리가 정의한 설정값들!
            "runtime_value": "I love LangGraph",
            "setting_value": 123
        }
    }
)
```

#### 🔍 핵심 포인트
- **"configurable"** 키 안에 모든 사용자 정의 설정을 넣어야 함
- ConfigSchema에서 정의한 키 이름과 **정확히 일치**해야 함
- 실행할 때마다 **다른 값**을 전달할 수 있음

In [ ]:
graph = builder.compile()
result = graph.invoke(
    {"messages": [HumanMessage(content="안녕하세요")]},
    config={
        "configurable": {"runtime_value": "I love LangGraph", "setting_value": 123}
    },
)

## 3.2 특수 노드: START와 END - 🚪 입구와 출구

모든 그래프에는 **시작점**과 **끝점**이 필요합니다! 마치 지하철 노선도의 **시발역**과 **종착역**처럼요.

### 🚇 지하철 노선도로 이해하기

- **🚪 START (시발역)**: 사용자 입력이 그래프로 들어오는 **첫 번째 문**
- **🏁 END (종착역)**: 모든 처리가 끝나고 결과가 나가는 **마지막 문**
- **🚉 중간역들**: 실제 작업을 처리하는 일반 노드들

### 🎯 START와 END의 특징

#### **START 노드** 🚀
- 그래프 실행이 **시작되는 지점**
- 사용자가 제공한 입력 데이터를 **첫 번째 작업 노드**로 전달
- 직접 정의할 필요 없음 (LangGraph가 자동 제공)

#### **END 노드** 🏁  
- 그래프 실행이 **완료되는 지점**
- 최종 결과를 **사용자에게 반환**
- 여러 노드에서 END로 연결 가능

### 💡 실제 사용 예시

```python
# 일반적인 플로우
builder.add_edge(START, "첫번째_작업")      # 🚪 → 🔨
builder.add_edge("마지막_작업", END)        # ⚙️ → 🏁

# 조건부 종료
builder.add_edge("검증_노드", END)          # ✅ → 🏁 (성공시)
builder.add_edge("재시도_노드", "첫번째_작업") # 🔄 → 🔨 (재시도시)
```

START와 END를 적절히 활용하면 **명확하고 이해하기 쉬운 그래프**를 만들 수 있어요! 🌟

In [ ]:
# START와 END 사용 예제
class FlowState(TypedDict):
    """State for flow control example"""

    value: int
    path: List[str]


builder = StateGraph(FlowState)


def start_processing(state: FlowState):
    """Initial processing node"""
    return {"value": state["value"] * 2, "path": ["start"]}


def middle_processing(state: FlowState):
    """Middle processing node"""
    return {"value": state["value"] + 10, "path": state["path"] + ["middle"]}


def final_processing(state: FlowState):
    """Final processing node"""
    return {"value": state["value"] // 2, "path": state["path"] + ["final"]}


# 노드 추가
builder.add_node("start_node", start_processing)
builder.add_node("middle_node", middle_processing)
builder.add_node("final_node", final_processing)

# 플로우 정의
builder.add_edge(START, "start_node")  # 진입점
builder.add_edge("start_node", "middle_node")
builder.add_edge("middle_node", "final_node")
builder.add_edge("final_node", END)  # 종료점

flow_graph = builder.compile()

# 실행
result = flow_graph.invoke({"value": 5, "path": []})
print(f"\n플로우 실행 결과:")
print(f"  초기값: 5")
print(f"  최종값: {result['value']}")
print(f"  경로: {' → '.join(result['path'])}")

## 3.3 노드 캐싱 - ⚡ 성능 최적화의 비밀 무기

**노드 캐싱**은 **한 번 계산한 결과를 저장해뒀다가 재사용**하는 똑똑한 기능입니다!

### 🥤 편의점 음료수 냉장고로 이해하기

편의점에서 음료수를 사는 과정을 생각해보세요:

#### 🏭 **캐싱 없을 때 (매번 새로 만들기)**
```
고객 요청 → 공장에서 제조 → 배송 → 고객에게 전달 (30분 소요)
```

#### ⚡ **캐싱 있을 때 (미리 만들어두기)**
```
고객 요청 → 냉장고에서 바로 꺼내기 → 고객에게 전달 (1분 소요)
```

### 🎯 CachePolicy 설정하기

#### **ttl (Time To Live)** ⏰
```python
ttl=60  # 60초 동안 캐시 유지
ttl=None  # 영원히 캐시 유지 (만료 없음)
```

#### **key_func (캐시 키 생성)** 🔑
```python
# 기본: 입력 전체를 키로 사용
key_func=None

# 커스텀: 특정 필드만 키로 사용
key_func=lambda state: hash(state["user_id"])
```

### 💡 언제 캐싱을 사용할까?

✅ **계산이 오래 걸리는 작업**: AI 모델 추론, 복잡한 데이터 처리  
✅ **결과가 자주 반복되는 경우**: 같은 질문에 대한 답변  
✅ **외부 API 호출**: 비용이 많이 드는 외부 서비스 호출  
✅ **데이터베이스 조회**: 자주 조회되는 정보들

### ⚠️ 주의사항

❌ **실시간 데이터**: 주식 가격, 날씨 등 계속 변하는 정보  
❌ **사용자별 개인화**: 각 사용자마다 다른 결과가 필요한 경우  
❌ **메모리 부족**: 캐시가 너무 많아지면 메모리 문제 발생 가능

실제 예제를 통해 캐싱의 놀라운 성능 향상을 체험해봅시다! 🚀

In [ ]:
import time
from langgraph.cache.memory import InMemoryCache
from langgraph.types import CachePolicy


class CacheState(TypedDict):
    """State for caching example"""

    x: int
    result: int


builder = StateGraph(CacheState)


def expensive_computation(state: CacheState) -> dict:
    """Expensive computation that we want to cache"""
    print(f"  🔄 무거운 계산 실행 중... (x={state['x']})")
    time.sleep(3)  # 무거운 작업 시뮬레이션
    return {"result": state["x"] * state["x"]}


# 캐시 정책과 함께 노드 추가
builder.add_node(
    "expensive_node",
    expensive_computation,
    cache_policy=CachePolicy(
        ttl=None,  # 60초 동안 캐시 유지, None 이면 만료시간 없음
        # key_func=lambda x: hash(x["x"])  # 커스텀 캐시 키 생성 함수
    ),
)

builder.add_edge(START, "expensive_node")
builder.add_edge("expensive_node", END)

# 캐시와 함께 컴파일
cached_graph = builder.compile(cache=InMemoryCache())

print("\n캐싱 테스트:")
print("첫 번째 실행")
start_time = time.time()
result1 = cached_graph.invoke({"x": 10})
print(f"  결과: {result1['result']}, 소요 시간: {time.time() - start_time:.1f}초")

print("\n두 번째 실행")
start_time = time.time()
result2 = cached_graph.invoke({"x": 10})
print(f"  결과: {result2['result']}, 소요 시간: {time.time() - start_time:.1f}초")

print("\n다른 입력으로 실행")
start_time = time.time()
result3 = cached_graph.invoke({"x": 20})
print(f"  결과: {result3['result']}, 소요 시간: {time.time() - start_time:.1f}초")

In [ ]:
print("\n캐싱 테스트:")

result1 = None
%time result1 = cached_graph.invoke({"x": 10})
print(f"  결과: {result1['result']}")

---

# Part 4: Edges (엣지) 심화 학습 🔀

**엣지(Edge)**는 그래프의 **교통신호등과 길안내**를 담당합니다! 현재 작업이 끝나면 다음에 어디로 가야 할지 알려주는 똑똑한 안내원이죠.

### 🚦 도시 교통 시스템으로 이해하기

도시의 교통 흐름을 떠올려보세요:

- **🚗 차량** = 데이터 (state)
- **🏢 건물들** = 노드들 (처리 장소)
- **🛣️ 도로** = 엣지들 (이동 경로)
- **🚦 신호등** = 라우팅 로직 (어디로 갈지 결정)

### 🎯 엣지의 핵심 역할

1. **🔗 연결**: 노드와 노드를 이어주는 다리 역할
2. **🧭 라우팅**: 상황에 따라 적절한 다음 노드 선택  
3. **⚡ 실행 제어**: 언제, 어떤 순서로 노드를 실행할지 결정
4. **🔄 흐름 관리**: 반복, 분기, 병합 등의 제어 흐름

### 📋 엣지의 4가지 종류

#### 1️⃣ **일반 엣지 (Normal Edges)** - 🚂 기차 노선
- **특징**: 항상 같은 경로로 이동
- **예시**: A → B → C (순서 고정)

#### 2️⃣ **조건부 엣지 (Conditional Edges)** - 🚦 교차로 신호
- **특징**: 상황에 따라 다른 경로 선택
- **예시**: 값이 크면 → 처리A, 작으면 → 처리B

#### 3️⃣ **매핑 엣지** - 🗺️ 내비게이션 시스템
- **특징**: 복잡한 조건을 미리 정의해둔 매핑 테이블
- **예시**: {"높음": "고급처리", "중간": "일반처리", "낮음": "간단처리"}

#### 4️⃣ **병렬 엣지** - 🚁 여러 헬기 동시 출발
- **특징**: 여러 노드를 동시에 실행
- **예시**: 하나의 작업을 여러 팀이 동시 처리

### 💡 언제 어떤 엣지를 사용할까?

- **일반 엣지**: 정해진 순서대로 처리할 때 (90% 케이스)
- **조건부 엣지**: 상황 판단이 필요할 때 (예: 성공/실패 분기)
- **매핑 엣지**: 복잡한 분류가 필요할 때 (예: 카테고리별 처리)
- **병렬 엣지**: 속도가 중요하거나 독립적인 작업들

이제 각각의 엣지 타입을 실제 예제로 살펴봅시다! 🚀

## 4.1 일반 엣지 (Normal Edges) - 🚂 정해진 노선 따라가기

**일반 엣지**는 가장 기본적인 형태로, **항상 같은 순서**로 노드를 실행합니다. 마치 지하철이 정해진 노선을 따라 운행하는 것과 같아요!

In [ ]:
class EdgeState(TypedDict):
    """State for edge examples"""

    value: int
    history: List[str]


builder = StateGraph(EdgeState)


# 노드 정의
def node_a(state: EdgeState):
    """Node A"""
    return {"value": state["value"] + 1, "history": state["history"] + ["A"]}


def node_b(state: EdgeState):
    """Node B"""
    return {"value": state["value"] * 2, "history": state["history"] + ["B"]}


def node_c(state: EdgeState):
    """Node C"""
    return {"value": state["value"] - 3, "history": state["history"] + ["C"]}


# 노드 추가
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)

# 일반 엣지 - 항상 A → B → C 순서로 실행
builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("b", "c")
builder.add_edge("c", END)

normal_edge_graph = builder.compile()

In [ ]:
# 그래프 시각화
visualize_graph(normal_edge_graph)

In [ ]:
result = normal_edge_graph.invoke({"value": 10, "history": []})
print(f"일반 엣지 결과:")
print(f"  경로: {' → '.join(result['history'])}")
print(f"  최종값: {result['value']}")

## 4.2 조건부 엣지 (Conditional Edges) - 🚦 똑똑한 신호등

**조건부 엣지**는 **현재 상황을 보고 어디로 갈지 결정**하는 똑똑한 신호등입니다!

### 🏥 응급실 트리아지로 이해하기

병원 응급실에서 환자를 분류하는 과정을 생각해보세요:

```
환자 도착 → 🩺 상태 확인 → 🚦 중증도 판단
                    ↓
         🔴 위급 → 중환자실
         🟡 보통 → 일반 진료실  
         🟢 경미 → 외래 진료실
```

### 🎯 조건부 엣지의 핵심 구조

```python
def routing_function(state):
    """상태를 보고 다음 노드 결정"""
    if state["urgency"] == "high":
        return "intensive_care"
    elif state["urgency"] == "medium": 
        return "general_care"
    else:
        return "outpatient_care"

# 조건부 엣지 추가
builder.add_conditional_edges(
    "triage_node",      # 출발 노드
    routing_function,   # 판단 함수
)
```

### 💡 조건부 엣지를 쓰면 좋은 경우

✅ **성공/실패 분기**: 작업 결과에 따른 다른 처리  
✅ **사용자 타입별 처리**: VIP, 일반, 신규 고객별 서비스  
✅ **데이터 크기별 처리**: 큰 파일과 작은 파일 다르게 처리  
✅ **오류 처리**: 에러 타입에 따른 복구 전략

In [ ]:
from typing import Literal

builder = StateGraph(EdgeState)

# 노드들 재사용
builder.add_node("a", node_a)
builder.add_node("b", node_b)
builder.add_node("c", node_c)


# 라우팅 함수 정의
def routing_function(state: EdgeState) -> Literal["b", "c"]:
    """Route based on state value"""
    if state["value"] > 15:
        return "b"  # 큰 값은 B로
    else:
        return "c"  # 작은 값은 C로


# 조건부 엣지 추가
builder.add_edge(START, "a")
builder.add_conditional_edges("a", routing_function)  # 소스 노드  # 라우팅 함수
builder.add_edge("b", END)
builder.add_edge("c", END)

conditional_graph = builder.compile()

# 다른 값으로 테스트
print("조건부 라우팅 테스트:\n")

for initial_value in [10, 20]:
    result = conditional_graph.invoke({"value": initial_value, "history": []})
    print(f"초기값 {initial_value}:")
    print(f"  경로: {' → '.join(result['history'])}")
    print(f"  최종값: {result['value']}\n")

## 4.3 매핑을 사용한 조건부 엣지 - 🗺️ 고급 내비게이션

**매핑 엣지**는 복잡한 조건들을 **미리 정리해둔 지도**입니다! 마치 내비게이션이 목적지별 최적 경로를 저장해두는 것과 같아요.

### 🍕 피자 배달 시스템으로 이해하기

피자 배달에서 지역별로 다른 배달원을 배치하는 시스템을 생각해보세요:

```python
def categorize_area(state):
    """주소를 보고 지역 분류"""
    address = state["address"]
    if "강남" in address or "서초" in address:
        return "premium_area"
    elif "마포" in address or "홍대" in address:
        return "trendy_area"  
    else:
        return "general_area"

# 매핑 테이블로 배달원 배정
area_mapping = {
    "premium_area": "vip_delivery",      # 고급 지역 → VIP 배달원
    "trendy_area": "young_delivery",     # 트렌디 지역 → 젊은 배달원
    "general_area": "standard_delivery"  # 일반 지역 → 일반 배달원
}

builder.add_conditional_edges(
    "order_received",    # 주문 접수 노드
    categorize_area,     # 지역 분류 함수
    area_mapping        # 📋 미리 정의된 매핑 테이블!
)
```

### 🎯 매핑 엣지의 장점

✅ **가독성**: 복잡한 if-else 대신 깔끔한 매핑 테이블  
✅ **유지보수**: 새로운 조건 추가가 쉬움  
✅ **명확성**: 모든 경우의 수가 한눈에 보임  
✅ **확장성**: 나중에 새로운 카테고리 추가 용이

### 💡 실제 활용 사례

- **🛒 쇼핑몰**: 상품 카테고리별 처리 프로세스
- **📧 이메일**: 발신자별 스팸/중요/일반 분류  
- **💼 인사관리**: 직급별 복지 혜택 시스템
- **🏦 은행**: 거래 금액별 승인 프로세스

In [ ]:
# 매핑을 사용한 더 복잡한 라우팅
builder = StateGraph(EdgeState)

builder.add_node(
    "start", lambda s: {"value": s["value"], "history": s["history"] + ["start"]}
)
builder.add_node(
    "process_high",
    lambda s: {"value": s["value"] * 10, "history": s["history"] + ["high"]},
)
builder.add_node(
    "process_medium",
    lambda s: {"value": s["value"] * 5, "history": s["history"] + ["medium"]},
)
builder.add_node(
    "process_low",
    lambda s: {"value": s["value"] * 2, "history": s["history"] + ["low"]},
)


def categorize_value(state: EdgeState) -> str:
    """Categorize value into high/medium/low"""
    if state["value"] > 100:
        return "high"
    elif state["value"] > 50:
        return "medium"
    else:
        return "low"


# 매핑을 사용한 조건부 엣지
builder.add_edge(START, "start")
builder.add_conditional_edges(
    "start",
    categorize_value,
    {"high": "process_high", "medium": "process_medium", "low": "process_low"},
)

# 모든 처리 노드는 END로
for node in ["process_high", "process_medium", "process_low"]:
    builder.add_edge(node, END)

mapped_graph = builder.compile()

# 테스트
print("매핑 기반 라우팅 테스트:\n")
for value in [30, 75, 150]:
    result = mapped_graph.invoke({"value": value, "history": []})
    print(f"초기값 {value}: {' → '.join(result['history'])}, 결과: {result['value']}")

## 4.4 병렬 실행 - 🚁 여러 헬기 동시 출격!

**병렬 실행**은 여러 작업을 **동시에 처리**해서 시간을 대폭 단축시키는 강력한 기능입니다!

### 🍳 요리 주방으로 이해하기

레스토랑 주방에서 요리하는 모습을 떠올려보세요:

#### 🐌 **순차 실행 (기존 방식)**
```
셰프1: 스프 만들기(10분) → 메인요리(15분) → 디저트(8분)
총 소요시간: 33분
```

#### ⚡ **병렬 실행 (똑똑한 방식)**
```
셰프1: 스프 만들기(10분)    ┐
셰프2: 메인요리(15분)       ├→ 가장 오래 걸리는 15분!
셰프3: 디저트(8분)         ┘
총 소요시간: 15분 (2배 빨라짐!)
```

### 🎯 병렬 실행이 일어나는 조건

**하나의 노드에서 여러 노드로 나가는 엣지**가 있으면 자동으로 병렬 실행됩니다!

```python
# 이렇게 하면 parallel_a, parallel_b, parallel_c가 동시 실행
def parallel_routing(state):
    return ["parallel_a", "parallel_b", "parallel_c"]

builder.add_conditional_edges("splitter", parallel_routing)
```

### 🚀 병렬 실행의 놀라운 장점

✅ **속도 향상**: 독립적인 작업들을 동시 처리로 시간 단축  
✅ **리소스 활용**: CPU 코어를 최대한 활용  
✅ **확장성**: 더 많은 작업을 동시에 처리 가능  
✅ **효율성**: 전체 시스템 처리량 대폭 향상

### 💡 병렬 실행이 효과적인 경우

- **🔍 정보 수집**: 여러 웹사이트에서 동시에 데이터 크롤링
- **🤖 AI 모델**: 여러 모델로 동시에 추론하고 결과 비교  
- **📊 데이터 처리**: 큰 데이터를 여러 조각으로 나누어 동시 처리
- **🌐 API 호출**: 여러 외부 서비스에 동시 요청

### ⚠️ 주의사항

❌ **의존성 있는 작업**: A 작업 결과가 B 작업에 필요한 경우  
❌ **공유 리소스**: 같은 파일이나 데이터베이스에 동시 접근  
❌ **순서 중요**: 실행 순서가 중요한 작업들

실제 예제에서 병렬 실행의 성능 향상을 체험해보세요! ⚡

In [ ]:
from typing import List


class ParallelState(TypedDict):
    """State for parallel execution"""

    input: int
    nodes: str
    results: Annotated[List[AnyMessage], add_messages]


builder = StateGraph(ParallelState)


def splitter(state: ParallelState):
    """Split into multiple paths"""
    return {"results": ["분할 완료"]}


def parallel_a(state: ParallelState):
    """Parallel processing A"""
    print("[Node A]")
    time.sleep(0.5)  # 작업 시뮬레이션
    return {"results": [f"A: {state['input'] * 2}"]}


def parallel_b(state: ParallelState):
    """Parallel processing B"""
    print("[Node B]")
    time.sleep(1.0)  # 작업 시뮬레이션
    return {"results": [f"B: {state['input'] + 10}"]}


def parallel_c(state: ParallelState):
    """Parallel processing C"""
    print("[Node C]")
    time.sleep(2.0)  # 작업 시뮬레이션
    return {"results": [f"C: {state['input'] ** 2}"]}


def merger(state: ParallelState):
    """Merge results"""
    return {"results": ["병합 완료"]}


# 노드 추가
builder.add_node("splitter", splitter)
builder.add_node("parallel_a", parallel_a)
builder.add_node("parallel_b", parallel_b)
builder.add_node("parallel_c", parallel_c)
builder.add_node("merger", merger)


# 병렬 실행을 위한 조건부 엣지
def parallel_routing(state: ParallelState) -> List[str]:
    """Route to multiple nodes for parallel execution"""

    print(f"[Parallel Routing] {state['nodes']}")

    if state["nodes"].upper() == "A":
        return ["parallel_a"]
    elif state["nodes"].upper() == "B":
        return ["parallel_b"]
    elif state["nodes"].upper() == "C":
        return ["parallel_c"]
    elif state["nodes"].upper() == "AB" or state["nodes"].upper() == "BA":
        return ["parallel_a", "parallel_b"]
    elif state["nodes"].upper() == "AC" or state["nodes"].upper() == "CA":
        return ["parallel_a", "parallel_c"]
    elif state["nodes"].upper() == "BC" or state["nodes"].upper() == "CB":
        return ["parallel_b", "parallel_c"]
    elif state["nodes"].upper() == "ABC":
        return ["parallel_a", "parallel_b", "parallel_c"]


builder.add_edge(START, "splitter")
builder.add_conditional_edges("splitter", parallel_routing)

# 모든 병렬 노드는 merger로
builder.add_edge("parallel_a", "merger")
builder.add_edge("parallel_b", "merger")
builder.add_edge("parallel_c", "merger")
builder.add_edge("merger", END)

parallel_graph = builder.compile()

In [ ]:
# 그래프 시각화
visualize_graph(parallel_graph, xray=True)

In [ ]:
from langchain_teddynote.messages import invoke_graph

start_time = time.time()
invoke_graph(
    parallel_graph, inputs={"input": 1, "nodes": "ca", "results": []}, config=None
)
elapsed = time.time() - start_time

print(f"\n실행 시간: {elapsed:.1f}초 (병렬 실행으로 시간 단축)")

---

# Part 5: Send - 동적 라우팅 시스템 📤

## 5.1 Send가 뭐고 왜 필요할까? 🤔

**Send**는 **실행 중에 동적으로** 여러 노드를 호출할 수 있는 강력한 도구입니다! 마치 **음식 배달 앱**에서 주문량에 따라 배달기사 수를 조절하는 것과 같아요.

### 🍕 배달 앱으로 이해하기

배달 앱에서 주문이 몰리는 상황을 떠올려보세요:

#### 🚶 **기존 방식 (고정적)**
```
주문 10개 → 배달기사 1명 → 10번 왕복 (100분 소요)
```

#### 🚁 **Send 방식 (동적)**
```
주문 10개 → 📦 주문 분석 → 🚀 배달기사 3명 동시 투입
               ↓
    기사1: 주문1,2,3 (30분)
    기사2: 주문4,5,6 (30분)  
    기사3: 주문7,8,9,10 (40분)
    총 소요시간: 40분 (2.5배 빨라짐!)
```

### 🎯 Send의 핵심 특징

#### **동적 노드 호출** 🎪
- 실행 시점에 **몇 개의 노드를 호출할지 결정**
- 각 노드에 **서로 다른 상태값** 전달 가능

#### **Map-Reduce 패턴** 🗺️
```
하나의 큰 작업 → 여러 작은 작업으로 분할 → 병렬 처리 → 결과 통합
```

#### **유연한 상태 관리** 🎭
- 메인 그래프와 **다른 상태 구조** 사용 가능
- 각 Send마다 **독립적인 데이터** 전달

### 💡 Send가 빛나는 상황들

✅ **대용량 데이터 처리**: 큰 파일을 여러 조각으로 나누어 병렬 처리  
✅ **동적 스케일링**: 작업량에 따라 처리 노드 수 자동 조절  
✅ **조건부 병렬화**: 상황에 따라 다른 수의 작업 생성  
✅ **배치 처리**: 여러 아이템을 각각 다른 설정으로 처리

### 🔧 Send 사용법 핵심

```python
def dynamic_router(state):
    sends = []
    for item in state["items"]:
        sends.append(Send("process_item", {"item": item, "id": item.id}))
    return sends

builder.add_conditional_edges("splitter", dynamic_router, ["process_item"])
```

이제 실제 예제를 통해 Send의 강력한 능력을 체험해봅시다! ✨

In [ ]:
from typing import TypedDict, List, Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.types import Send
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import random

# ChatGPT 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)


# State 정의
class JokeGeneratorState(TypedDict):
    """농담 생성 상태"""

    jokes: Annotated[List[AnyMessage], add_messages]
    current_subject: str
    attempt_count: int


class SingleJokeState(TypedDict):
    """개별 농담 생성 상태"""

    subject: str
    joke_number: int


# StateGraph 생성
builder = StateGraph(JokeGeneratorState)


def initialize_state(state: JokeGeneratorState) -> dict:
    """초기 상태 설정"""
    subjects = ["프로그래머", "AI", "파이썬", "자바스크립트", "데이터베이스"]
    selected_subject = random.choice(subjects)

    print(f"🎯 선택된 주제: {selected_subject}")
    print("=" * 50)

    return {"current_subject": selected_subject, "jokes": [], "attempt_count": 0}


def generate_single_joke(state: SingleJokeState) -> dict:
    """LLM을 사용하여 개별 농담 생성"""
    messages = [
        SystemMessage(
            content="당신은 재미있는 IT 농담을 만드는 코미디언입니다. 짧고 재치있는 농담을 한국어로 만들어주세요."
        ),
        HumanMessage(
            content=f"{state['subject']}에 대한 재미있는 농담을 하나만 만들어주세요. (농담 #{state['joke_number']})"
        ),
    ]

    response = llm.invoke(messages)
    joke = response.content.strip()

    print(f"🎭 농담 #{state['joke_number']}: {joke}")

    return {"jokes": [joke]}


def update_attempt_count(state: JokeGeneratorState) -> dict:
    """시도 횟수 업데이트"""
    attempt_count = state.get("attempt_count", 0) + 1
    current_joke_count = len(state.get("jokes", []))

    print(f"\n📊 현재 농담 개수: {current_joke_count}/3")
    print(f"🔄 시도 횟수: {attempt_count}")

    return {"attempt_count": attempt_count}


def route_based_on_count(state: JokeGeneratorState) -> List[Send]:
    """농담 개수에 따라 Send로 라우팅"""
    current_joke_count = len(state.get("jokes", []))

    if current_joke_count < 3:
        # 3개 미만이면 부족한 만큼 generate_joke로 Send
        sends = []
        for i in range(current_joke_count + 1, 4):  # 3개까지 생성
            sends.append(
                Send(
                    "generate_joke",
                    {"subject": state["current_subject"], "joke_number": i},
                )
            )
        print(f"➡️ {len(sends)}개의 농담을 추가로 생성합니다...")
        return sends
    else:
        # 3개 이상이면 finalize로
        print("✅ 농담 3개 생성 완료! 최종 정리 단계로 이동합니다.")
        return [Send("finalize", state)]


def finalize_jokes(state: JokeGeneratorState) -> dict:
    """최종 농담 정리"""
    print("\n" + "=" * 50)
    print(f"🎉 최종 농담 컬렉션 ({state['current_subject']} 주제)")
    print("=" * 50)

    jokes = state.get("jokes", [])
    for i, joke in enumerate(jokes, 1):
        print(f"\n농담 {i}: {joke}")

    summary = f"\n\n📝 총 {len(jokes)}개의 농담이 생성되었습니다. (시도 횟수: {state.get('attempt_count', 1)}회)"

    return {"jokes": jokes + [summary]}


# 노드 추가
builder.add_node("initialize", initialize_state)
builder.add_node("generate_joke", generate_single_joke)
builder.add_node("update_count", update_attempt_count)
builder.add_node("finalize", finalize_jokes)

# 엣지 추가
builder.add_edge(START, "initialize")
builder.add_edge("initialize", "update_count")

# update_count 노드 이후 conditional_edges로 Send 처리
builder.add_conditional_edges(
    "update_count",
    route_based_on_count,  # Send 리스트를 반환하는 라우팅 함수
    ["generate_joke", "finalize"],  # 가능한 목적지 노드들
)

# generate_joke 완료 후 다시 update_count로
builder.add_edge("generate_joke", "update_count")

# finalize 후 종료
builder.add_edge("finalize", END)

# 그래프 컴파일
joke_generator = builder.compile()

In [ ]:
from langchain_teddynote.messages import stream_graph

stream_graph(
    joke_generator,
    inputs={"current_subject": "AI", "jokes": [], "attempt_count": 0},
    config=None,
)

---

# Part 6: Command - 상태 업데이트와 제어 흐름의 통합 ⚙️

## 6.1 Command가 뭐고 왜 특별할까? 🤔

**Command**는 **한 번에 두 가지 일**을 할 수 있는 다기능 도구입니다! 상태를 업데이트하면서 동시에 다음 노드를 결정할 수 있어요.

### 🎛️ 업무 처리 시스템으로 이해하기

회사에서 업무를 처리하는 상황을 떠올려보세요:

#### 🎯 **기존 방식 (두 단계 분리)**
```
1단계: 리소스 할당 (상태 업데이트)
2단계: 다음 행동 결정 (제어 흐름)
```

#### ⚡ **Command 방식 (한 번에 처리)**
```
업무 처리 → Command {
    예산 -30000,           ← 상태 업데이트
    진행률 +50,            ← 상태 업데이트  
    goto: "프로젝트_계속"   ← 제어 흐름
}
```

### 🎯 Command의 두 가지 핵심 능력

#### 1️⃣ **update** - 상태 업데이트 📝
```python
Command(
    update={
        "score": state["score"] + 100,
        "level": "advanced",
        "items": state["items"] + ["sword"]
    }
)
```

#### 2️⃣ **goto** - 제어 흐름 🚀
```python
Command(
    update={...},
    goto="next_stage"  # 다음에 실행할 노드 지정
)
```

### 💡 Command의 활용 상황

✅ **비즈니스 로직**: 작업과 상태 변화를 동시에 처리  
✅ **워크플로우**: 작업 완료와 동시에 다음 단계 결정  
✅ **에이전트 핸드오프**: 정보 전달과 함께 다른 에이전트로 이동  
✅ **조건부 처리**: 결과에 따라 상태 업데이트와 경로 분기

### 🤝 Command가 해결하는 문제

실제 개발에서는 **상태 업데이트와 제어 흐름을 동시에** 처리해야 할 때가 많습니다.

#### 🏥 **병원 응급실 예시**
```python
환자 진료 완료 → Command {
    update: {
        "treatment_status": "완료",
        "next_appointment": "2024-01-15",
        "bill_amount": 50000
    },
    goto: "discharge" if 완치 else "follow_up"
}
```

Command를 사용하면 **복잡한 비즈니스 로직을 간단하고 명확하게** 표현할 수 있습니다!

In [ ]:
from langgraph.types import Command
from typing import Literal
from typing_extensions import TypedDict


class CommandState(TypedDict):
    """State for Command examples"""

    value: int


def my_node(state: CommandState) -> Command[Literal["my_other_node"]]:
    """Node that makes decisions and updates state"""
    return Command(
        # state update
        update={"foo": "bar"},
        # control flow
        goto="my_other_node",
    )

Command를 사용하면 동적 제어 흐름 동작(조건부 에지와 동일)을 구현할 수도 있습니다.

In [ ]:
def my_node(state: CommandState) -> Command[Literal["my_other_node"]]:
    if state["foo"] == "bar":
        return Command(update={"foo": "baz"}, goto="my_other_node")

**상위 그래프의 노드로 이동하기**

하위 그래프를 사용하는 경우 하위 그래프 내의 노드에서 다른 하위 그래프(즉, 상위 그래프의 다른 노드)로 이동하고 싶을 수 있습니다. 이렇게 하려면 Command에서 graph=Command.PARENT를 지정하면 됩니다.

In [ ]:
def my_node(state: CommandState) -> Command[Literal["other_subgraph"]]:
    return Command(
        update={"foo": "bar"},
        goto="other_subgraph",  # where `other_subgraph` is a node in the parent graph
        graph=Command.PARENT,
    )

`Command` 를 사용하여 `update` 와 `goto` 를 동시에 사용하는 예제

In [ ]:
from langgraph.types import Command


class CommandState(TypedDict):
    """State for Command examples"""

    value: int
    message: str
    path: List[str]


builder = StateGraph(CommandState)


# Command를 반환하는 노드
def decision_node(
    state: CommandState,
) -> Command[Literal["success", "failure", "retry"]]:
    """Node that makes decisions and updates state"""
    value = state["value"]

    if value > 100:
        return Command(
            # 상태 업데이트
            update={
                "message": "값이 너무 큽니다",
                "path": state["path"] + ["decision"],
            },
            # 라우팅
            goto="failure",
        )
    elif value > 50:
        return Command(
            update={
                "message": "처리 성공",
                "value": value * 2,
                "path": state["path"] + ["decision"],
            },
            goto="success",
        )
    else:
        return Command(
            update={
                "message": "재시도 필요",
                "value": value + 30,
                "path": state["path"] + ["decision"],
            },
            goto="retry",
        )


def success_node(state: CommandState):
    """Success handler"""
    return {"message": f"✅ {state['message']}", "path": state["path"] + ["success"]}


def failure_node(state: CommandState):
    """Failure handler"""
    return {"message": f"❌ {state['message']}", "path": state["path"] + ["failure"]}


def retry_node(state: CommandState) -> Command[Literal["decision"]]:
    """Retry handler - goes back to decision"""
    return Command(
        update={"message": "재시도 중...", "path": state["path"] + ["retry"]},
        goto="decision",  # 다시 결정 노드로
    )


# 노드 추가
builder.add_node("decision", decision_node)
builder.add_node("success", success_node)
builder.add_node("failure", failure_node)
builder.add_node("retry", retry_node)

# 엣지 추가
builder.add_edge(START, "decision")
builder.add_edge("success", END)
builder.add_edge("failure", END)
# retry는 Command로 decision으로 돌아감

command_graph = builder.compile()

In [ ]:
visualize_graph(command_graph)

In [ ]:
# 그래프 실행
test_values = [30, 75, 150]
for val in test_values:
    result = command_graph.invoke({"value": val, "message": "", "path": []})
    print(f"초기값 {val}:")
    print(f"  경로: {' → '.join(result['path'])}")
    print(f"  메시지: {result['message']}")
    print(f"  최종값: {result['value']}\n")

## 6.1 Command vs 조건부 엣지

### 언제 Command를 사용할까?
- 상태 업데이트와 라우팅을 **동시에** 수행해야 할 때
- 멀티 에이전트 핸드오프에서 정보 전달이 필요할 때

### 언제 조건부 엣지를 사용할까?
- 상태 업데이트 없이 **라우팅만** 필요할 때
- 단순한 분기 로직

In [ ]:
# 멀티 에이전트 핸드오프 예제
class AgentState(TypedDict):
    """State for multi-agent system"""

    task: str
    current_agent: str
    result: str
    handoff_info: dict


builder = StateGraph(AgentState)


def analyst_agent(state: AgentState) -> Command[Literal["engineer", "designer"]]:
    """Analyst agent that delegates tasks"""
    task = state["task"]

    if "코드" in task or "구현" in task:
        return Command(
            update={
                "current_agent": "engineer",
                "handoff_info": {
                    "priority": "high",
                    "language": "Python",
                    "analyzed_by": "analyst",
                },
            },
            goto="engineer",
        )
    else:
        return Command(
            update={
                "current_agent": "designer",
                "handoff_info": {"style": "modern", "analyzed_by": "analyst"},
            },
            goto="designer",
        )


def engineer_agent(state: AgentState):
    """Engineer agent"""
    info = state["handoff_info"]
    return {"result": f"엔지니어가 {info.get('language', 'unknown')}로 구현 완료"}


def designer_agent(state: AgentState):
    """Designer agent"""
    info = state["handoff_info"]
    return {"result": f"디자이너가 {info.get('style', 'unknown')} 스타일로 디자인 완료"}


# 노드 추가
builder.add_node("analyst", analyst_agent)
builder.add_node("engineer", engineer_agent)
builder.add_node("designer", designer_agent)

# 엣지 추가
builder.add_edge(START, "analyst")
builder.add_edge("engineer", END)
builder.add_edge("designer", END)

agent_graph = builder.compile()

In [ ]:
visualize_graph(agent_graph)

In [ ]:
# 테스트
print("멀티 에이전트 핸드오프 테스트:\n")

tasks = ["로그인 기능 구현", "홈페이지 디자인"]
for task in tasks:
    result = agent_graph.invoke(
        {"task": task, "current_agent": "analyst", "result": "", "handoff_info": {}}
    )
    print(f"작업: {task}")
    print(f"  담당 에이전트: {result['current_agent']}")
    print(f"  전달 정보: {result['handoff_info']}")
    print(f"  결과: {result['result']}\n")

---

# Part 7: 고급 기능 🚀

## 재귀 제한 (Recursion Limit)

무한 루프를 방지하기 위해 그래프 실행의 최대 super-step 수를 제한할 수 있습니다.

In [ ]:
class LoopState(TypedDict):
    """State for loop example"""

    counter: int
    history: List[int]


builder = StateGraph(LoopState)


def increment_node(state: LoopState) -> Command[Literal["increment", "end"]]:
    """Node that increments counter"""
    new_counter = state["counter"] + 1

    print(f"new_counter: {new_counter}")

    if new_counter < 10:  # 의도적으로 높은 목표 설정
        return Command(
            update={
                "counter": new_counter,
                "history": state["history"] + [new_counter],
            },
            goto="increment",  # 자기 자신으로 루프
        )
    else:
        return Command(update={"counter": new_counter}, goto="end")


def end_node(state: LoopState):
    """End node"""
    return {"history": state["history"] + [999]}


builder.add_node("increment", increment_node)
builder.add_node("end", end_node)
builder.add_edge(START, "increment")
builder.add_edge("end", END)

loop_graph = builder.compile()

# 재귀 제한 설정
print("재귀 제한 테스트:\n")

try:
    # 재귀 제한 5로 실행
    result = loop_graph.invoke(
        {"counter": 0, "history": []}, config={"recursion_limit": 5}  # 최대 5번만 실행
    )
except Exception as e:
    print(f"❌ 에러 발생: {e}")

In [ ]:
# 충분한 재귀 제한으로 실행
result = loop_graph.invoke(
    {"counter": 0, "history": []}, config={"recursion_limit": 100}
)
print(f"\n✅ 성공: counter={result['counter']}, 실행 횟수={len(result['history'])}")